# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("/Users/nathalylamas/Downloads/Starter_Code/output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,enewetak,11.3474,162.3373,28.56,81,100,6.95,MH,1726451505
1,1,arraial do cabo,-22.9661,-42.0278,21.70,92,0,5.12,BR,1726451506
2,2,bengkulu,-3.8004,102.2655,27.68,72,0,0.53,ID,1726451507
3,3,port-aux-francais,-49.3500,70.2167,2.74,79,100,14.69,TF,1726451508
4,4,waitangi,-43.9535,-176.5597,11.01,83,59,0.45,NZ,1726451455


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    'Lng', 'Lat', geo=True, size='Humidity', 
    color='Humidity', cmap='Viridis', alpha=0.7, 
    tiles='OSM', frame_width=700, frame_height=500, 
    hover_cols=['City', 'Country', 'Humidity']
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity,City,Country)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [10]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_df = city_data_df[
    (city_data_df["Max Temp"] > 22) & 
    (city_data_df["Max Temp"] < 27) & 
    (city_data_df["Wind Speed"] < 5) & 
    (city_data_df["Cloudiness"] == 0)
]
# Drop any rows with null values
ideal_weather_df = ideal_weather_df.dropna()

# Display sample data
ideal_weather_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
46,46,marquette,46.5333,-87.6335,22.80,75,0,1.79,US,1726451558
134,134,hamilton,39.1834,-84.5333,24.45,48,0,2.06,US,1726451659
195,195,siparia,10.1333,-61.5000,26.43,85,0,3.67,TT,1726451729
260,260,east saint louis,38.6245,-90.1509,24.17,92,0,2.06,US,1726451805
262,262,brookings,44.3114,-96.7984,24.00,74,0,4.12,US,1726451807


### Step 3: Create a new DataFrame called `hotel_df`.

In [11]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
46,marquette,US,46.5333,-87.6335,75,
134,hamilton,US,39.1834,-84.5333,48,
195,siparia,TT,10.1333,-61.5000,85,
260,east saint louis,US,38.6245,-90.1509,92,
262,brookings,US,44.3114,-96.7984,74,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [12]:
# Set parameters to search for a hotel
radius = 10000
params = {"categories": "accommodation.hotel",
    "apiKey": geoapify_key,               
    "limit": 1 
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address =  name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
marquette - nearest hotel: No hotel found
hamilton - nearest hotel: North Vista Manor
siparia - nearest hotel: Double Palms
east saint louis - nearest hotel: Casino Queen Hotel
brookings - nearest hotel: Quality Inn
ilha de mocambique - nearest hotel: Ruby Backpackers
mingshui - nearest hotel: No hotel found
turpan - nearest hotel: Oriental
matoury - nearest hotel: La Marmotte
archangelos - nearest hotel: Semina


,City,Country,Lat,Lng,Humidity,Hotel Name
46,marquette,US,46.5333,-87.6335,75,No hotel found
134,hamilton,US,39.1834,-84.5333,48,North Vista Manor
195,siparia,TT,10.1333,-61.5000,85,Double Palms
260,east saint louis,US,38.6245,-90.1509,92,Casino Queen Hotel
262,brookings,US,44.3114,-96.7984,74,Quality Inn
274,ilha de mocambique,MZ,-15.0342,40.7358,73,Ruby Backpackers
279,mingshui,CN,36.7167,117.5000,54,No hotel found
374,turpan,CN,42.9333,89.1667,22,Oriental
444,matoury,GF,4.8483,-52.3317,89,La Marmotte
450,archangelos,GR,36.2149,28.1149,57,Semina


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [13]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    'Lng', 'Lat', geo=True, size='Humidity', 
    color='Humidity', cmap='Viridis', alpha=0.7, 
    tiles='OSM', frame_width=700, frame_height=500, 
    hover_cols=['City', 'Country', 'Hotel Name', 'Humidity']
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity,City,Country,Hotel Name)